In [157]:
import requests
import urllib.request
import pickle
from bs4 import BeautifulSoup
import csv
import os
import string


In [159]:
def getResponse(letter):
    url = 'https://www.sup.org/search/authors/?letter=' + letter
    response = requests.get(url)
    if response:
        print("yay! page download for " + letter +" : ok!")
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [160]:
def addNames(soupObj):
    authors = soupObj.findAll('div', class_="author-browse")
    for au in authors:
        listChildren = list(au.children)
        a = listChildren[0].strip()
        if len(a) > 0: #has author name
            uniqueNames.append([a,listChildren[3].get_text()])  #adds to global list
        elif len(a) == 0: #
            uniqueNames[len(uniqueNames)-1].append(listChildren[1].get_text())

In [162]:
def saveAllAuthorsCSV_temp():
    with open('./authorNames.csv', 'w', newline='') as csvfile:
        authorWriter = csv.writer(csvfile, delimiter='\t')
        for au in uniqueNames:
            authorWriter.writerow(au)

In [163]:
def pickleAuthors():
    pickle.dump(uniqueNames, open("./authorPickle", "wb"))

In [164]:

###run every couple hours?
def unpickleAuthors():
    names = pickle.load(open("./authorPickle", "rb"))

In [165]:
def getSplit(splitSize):
    totalLength = len(names)
    splits = [(x,x+splitSize) 
              if x+splitSize < 4150 
              else (x, 4125) 
              for x in range(0, totalLength, splitSize)]
    return splits

In [172]:
def saveSplitCSVs(splits):
    fileBase = './authorNameSplit'
    for i,s in enumerate(splits):
        file = fileBase + str(i) + '.csv'
        with open(file, 'w', newline='') as csvfile:
            authorWriter = csv.writer(csvfile, delimiter='\t')
            for au in names[s[0]:s[1]]:
                authorWriter.writerow(au)
        

In [191]:
def retrievePopulate(splitNo):
    fileBase = './authorNameSplit'
    file = fileBase + str(splitNo) + '.csv'
    
    #generate files
    finalBase = './authorsCSV'
    final = finalBase + str(splitNo) + '.csv'
    finalCSV = open(final, "w+") ##CREATE new file to populate with CSV info
    finalCSVwriter = csv.writer(finalCSV, delimiter='\t')
    #populate it in the following format:
    #AUTHOR TITLE YEAR CITATION
    
    errorBase = './errorsCSV'
    error = errorBase + str(splitNo) + '.csv'
    errorCSV = open(error, "w+") ##CREATE new file for ones with errors
    errorCSVwriter = csv.writer(errorCSV, delimiter='\t')
    
    
    print(file)
    with open(file, 'r') as csvfile:
        authorReader = csv.reader(csvfile, delimiter='\t')
        for line in authorReader:
            if len(line) <= 2:
                #just one book
                author = line[0]
                book = line[1]
                write2CSV(author, book, finalCSVwriter, errorCSVwriter)
            elif len(line) > 2:
                author = line[0]
                for b in range(len(line)-1):
                    write2CSV(author, line[b+1], finalCSVwriter, errorCSVwriter)
                #more than one book
    
    #close both files
    close(final)
    close(error)

In [192]:
def write2CSV(author, book, finalCSVwriter, errorCSVwriter): #input list form
    book = book.lower().translate(str.maketrans('', '', string.punctuation))
    print(book)
    year, cites = retrieveScholarInfo(author, book)
    finalCSVwriter.writerow([author, book, year, cites])
    if year==None and cities==None:
        errorCSVwriter.writerow([author, book])

In [196]:
def retrieveScholarInfo(name, book):
    var = !python scholar.py --author $name --phrase $book --pub "Stanford University Press" --csv 
    print(var)
    if 'Traceback' in var[0]: #exception handling: write to error csv
        return None, None
    print(var[0])
    return var[0].split("|")[2], var[0].split("|")[3]
    

In [194]:
##### MAIN: setup ####


alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
uniqueNames = []
#remove prev files: 
!rm *.csv
#loop through all alphabet chars
for char in alpha:
    soup = getResponse(char)
    addNames(soup)
pickleAuthors()
saveAllAuthorsCSV_temp()
splits = getSplit(10)
saveSplitCSVs(splits)


yay! page download for A : ok!
yay! page download for B : ok!
yay! page download for C : ok!
yay! page download for D : ok!
yay! page download for E : ok!
yay! page download for F : ok!
yay! page download for G : ok!
yay! page download for H : ok!
yay! page download for I : ok!
yay! page download for J : ok!
yay! page download for K : ok!
yay! page download for L : ok!
yay! page download for M : ok!
yay! page download for N : ok!
yay! page download for O : ok!
yay! page download for P : ok!
yay! page download for Q : ok!
yay! page download for R : ok!
yay! page download for S : ok!
yay! page download for T : ok!
yay! page download for U : ok!
yay! page download for V : ok!
yay! page download for W : ok!
yay! page download for X : ok!
yay! page download for Y : ok!
yay! page download for Z : ok!


In [197]:
##### MAIN: piecemeal grab ####

splitNo = 0
retrievePopulate(splitNo)


./authorNameSplit0.csv
marine algae of california
[]


IndexError: list index out of range